## load_raw_data

In [23]:
import numpy as np
import pandas as pd
import os
from dotenv import load_dotenv
from colorama import Fore, Style

In [24]:
from params import RAW_DATA_PATH, TEXT_FILE, RATES_FILE

In [27]:
######## load_raw_data ########
# Description: import raw from a local folder into a dataframe
# Args: folder path
# Kwargs: N/A
# Seps: defines local folder path as variable
#       pull data using path variable
# Output : tuple with two dataframes 

def load_raw_data():

    # Load environment variables from .env file
    load_dotenv(override=True)
    
    print(Fore.MAGENTA + "\nLoading raw data..." + Style.RESET_ALL)
    
    # Debugging: Check if environment variables are loaded correctly
    print(f"RAW_DATA_PATH: {RAW_DATA_PATH}")
    print(f"TEXT_FILE: {TEXT_FILE}")
    print(f"RATES_FILE: {RATES_FILE}")
    
    # Construct full file paths
    text_data = os.path.join(RAW_DATA_PATH, TEXT_FILE)
    rates_data = os.path.join(RAW_DATA_PATH, RATES_FILE)

     # Debugging: Check if paths exist
    if not os.path.exists(text_data):
        raise FileNotFoundError(f"Error: The text file was not found at {text_data}")

    if not os.path.exists(rates_data):
        raise FileNotFoundError(f"Error: The rates file was not found at {rates_data}")

    
    text_df = pd.read_csv(text_data)
    rates_df = pd.read_csv(rates_data)
    
    print(f"Data loaded from {text_data, rates_data}")
    
    return text_df, rates_df

In [28]:
text_df, rates_df = load_raw_data()

print(text_df.head(), rates_df)

Python-dotenv could not parse statement starting at line 1



Loading raw data...
RAW_DATA_PATH: /home/antonio/code/aferri-git/FED-Predictor/data/raw
TEXT_FILE: Fed_Scrape-2015-2023.csv
RATES_FILE: US Fed Rate.csv
Data loaded from ('/home/antonio/code/aferri-git/FED-Predictor/data/raw/Fed_Scrape-2015-2023.csv', '/home/antonio/code/aferri-git/FED-Predictor/data/raw/US Fed Rate.csv')
   Unnamed: 0      Date  Type  \
0           0  20230412     0   
1           1  20230412     0   
2           2  20230412     0   
3           3  20230412     0   
4           4  20230412     0   

                                                Text  
0  The Federal Reserve on Wednesday released the ...  
1  The minutes for each regularly scheduled meeti...  
2  The minutes can be viewed on the Board's website.  
3  For media inquiries, e-mail [email protected] ...  
4  Minutes of the Federal Open Market Committee\r...        Release Date   Time Actual Forecast Previous
0    Nov 01, 2023  13:00  5.50%    5.50%    5.50%
1    Sep 20, 2023  13:00  5.50%    5.50%    5.5

## adjust_column_names

In [29]:
def adjust_column_names(df, rename_dict=None):
    
    df.columns = df.columns.str.lower()
    
    if rename_dict:  # Only rename if a valid dictionary is provided
        df = df.rename(columns=rename_dict)

    return df

In [30]:
text_df = adjust_column_names(text_df)
rates_df= adjust_column_names(rates_df, {'release date' : 'date', 'actual' : 'rate'})

In [31]:
print(text_df.columns)
print(rates_df.columns)

Index(['unnamed: 0', 'date', 'type', 'text'], dtype='object')
Index(['date', 'time', 'rate', 'forecast', 'previous'], dtype='object')


## format_raw_data

In [32]:
def format_raw_data(
    text_df, rates_df, 
    date='date',
    rate='rate'
):

    try:
        text_df[date] = pd.to_datetime(text_df[date], format='%Y%m%d')
        rates_df[date] = pd.to_datetime(rates_df[date], format='%b %d, %Y')  # Note: specific format for rate dates
    except Exception as e:
        raise ValueError(f"Error while converting date columns: {e}")
    
    rates_df[rate] = rates_df[rate].str.rstrip('%').astype(float)

    start_date_text_df = text_df[date].min()
    rates_df = rates_df[rates_df[date] >= start_date_text_df]

    return text_df, rates_df

In [33]:
text_df, rates_df = format_raw_data(text_df, rates_df)

In [34]:
print(text_df.info(), rates_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9974 entries, 0 to 9973
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   unnamed: 0  9974 non-null   int64         
 1   date        9974 non-null   datetime64[ns]
 2   type        9974 non-null   int64         
 3   text        9974 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 311.8+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 72 entries, 0 to 71
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      72 non-null     datetime64[ns]
 1   time      72 non-null     object        
 2   rate      72 non-null     float64       
 3   forecast  72 non-null     object        
 4   previous  72 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 3.4+ KB
None None


## sort_dates

In [35]:
def sort_dates(df, df_type, reference_df=None):
    
    # Ensure 'date' is a column (if already an index, reset it)
    if 'date' in df.index:
        df = df.reset_index()
    
    df = df.sort_values(by='date')  # Sort by date
    
    # If df_type is 'rates', filter based on reference_df (text_df)
    if df_type == 'rates' and reference_df is not None:
        start_date = reference_df['date'].min()  # Get earliest date from reference_df
        df = df[df['date'] >= start_date]  # Filter df where date is >= start_date

    return df

In [36]:
text_df = sort_dates(text_df, 'text')
rates_df = sort_dates(rates_df, 'rates', text_df)

In [37]:
print(text_df.head())

      unnamed: 0       date  type  \
9973        9973 2015-01-07     0   
9969        9969 2015-01-07     0   
9972        9972 2015-01-07     0   
9971        9971 2015-01-07     0   
9970        9970 2015-01-07     0   

                                                   text  
9973  \r\n       For media inquiries, call 202-452-2...  
9969  \r\n       The Federal Reserve Board and the F...  
9972  \nMinutes of the Federal Open Market Committee...  
9971  \r\n       FOMC minutes can be viewed on the B...  
9970  \r\n       The minutes for each regularly sche...  


In [38]:
print(rates_df.head())

         date   time  rate forecast previous
71 2015-01-28  14:00  0.25    0.25%    0.25%
70 2015-03-18  13:00  0.25    0.25%    0.25%
69 2015-04-29  13:00  0.25    0.25%    0.25%
68 2015-06-17  13:00  0.25    0.25%    0.25%
67 2015-07-29  13:00  0.25    0.25%    0.25%


## text_encode

In [39]:
def text_encode(df, column, df_type):
    
    # Ensure the column exists before applying any transformation
    if column not in df.columns:
        raise KeyError(f"Column '{column}' does not exist in the {df_type} dataframe.")
    
    if df_type == 'text': # check what df_type it is
        df['type_text'] = df[column].apply(lambda x: 'statement' if x == 0 else 'minutes')
    
    elif df_type == 'rates':
        df['rate_change'] = df[column].diff()  # Calculate the difference in rate values
        df['rate_change_text'] = df['rate_change'].apply(lambda x: 'up' if x > 0 else ('down' if x < 0 else 'no change')).astype(str)
        
    else:
        raise ValueError("Invalid df_type. Choose 'text' or 'rates'.")

    print(f"{df_type} dataframe after encoding:\n", df.head())
    return df

In [42]:
text_df = text_encode(text_df, 'type', 'text')
rates_df = text_encode (rates_df, 'rate', 'rates')

text dataframe after encoding:
       unnamed: 0       date  type  \
9973        9973 2015-01-07     0   
9969        9969 2015-01-07     0   
9972        9972 2015-01-07     0   
9971        9971 2015-01-07     0   
9970        9970 2015-01-07     0   

                                                   text  type_text  
9973  \r\n       For media inquiries, call 202-452-2...  statement  
9969  \r\n       The Federal Reserve Board and the F...  statement  
9972  \nMinutes of the Federal Open Market Committee...  statement  
9971  \r\n       FOMC minutes can be viewed on the B...  statement  
9970  \r\n       The minutes for each regularly sche...  statement  
rates dataframe after encoding:
          date   time  rate forecast previous  rate_change rate_change_text
71 2015-01-28  14:00  0.25    0.25%    0.25%          NaN        no change
70 2015-03-18  13:00  0.25    0.25%    0.25%          0.0        no change
69 2015-04-29  13:00  0.25    0.25%    0.25%          0.0        no chang

## sliding_window and group_text

In [43]:
from datetime import timedelta

In [44]:
def group_text(rate_date, text_df, date_diff):
    window_size = timedelta(days=date_diff)

    # Filter texts that occurred before the rate decision
    valid_texts = text_df[text_df['date'] < rate_date]

    # Apply sliding window: Get texts within the specified window size before rate_date
    texts_in_window = valid_texts[valid_texts['date'] >= rate_date - window_size]

    # Combine the texts
    grouped_texts = ' '.join(texts_in_window['text'])
    
    return grouped_texts


In [45]:
def sliding_window(rates_df, text_df):
    
    rates_df = rates_df.copy()
    
    # Calculate the difference between consecutive rate decisions to determine dynamic window size
    rates_df['next_date'] = rates_df['date'].shift(-1)
    rates_df['date_diff'] = (rates_df['next_date'] - rates_df['date']).dt.days
    
    # Corrects for NaNs since we are subtracting time deltas
    rates_df['date_diff'] = rates_df['date_diff'].fillna(0).astype(int)

    # isolate statements and minutes as they occurr at different times relative to previous decisions
    statement_df = text_df[text_df['type_text'] == 'statement']
    minutes_df = text_df[text_df['type_text'] == 'minutes']    

    pairing_data  = []

    for _, rate_row in rates_df.iterrows():
        rate_date = rate_row['date']
        rate = rate_row['rate_change_text']
        date_diff = rate_row['date_diff']
        
        grouped_statements = group_text(rate_date, statement_df, date_diff)
        grouped_minutes = group_text(rate_date, minutes_df, date_diff)
    
        # Add the data to pairing_df
        pairing_data.append({
            'decision': rate,
            'date': rate_date,
            'grouped_statements': grouped_statements,
            'grouped_minutes': grouped_minutes,
            'window_size_days': date_diff  # Store the dynamic window size for reference
        })

    pairing_df = pd.DataFrame(pairing_data)
    pairing_df = pairing_df.set_index("date")
    
    return pairing_df

In [46]:
pairing_df = sliding_window(rates_df, text_df)
print(pairing_df.head())

             decision                                 grouped_statements  \
date                                                                       
2015-01-28  no change  \r\n       For media inquiries, call 202-452-2...   
2015-03-18  no change  \nSubmission of Tender\nParticipants must subm...   
2015-04-29  no change  \r\n       Information received since the Fede...   
2015-06-17  no change  \r\n       The Federal Open Market Committee o...   
2015-07-29  no change  \r\n      Voting for the FOMC monetary policy ...   

                                              grouped_minutes  \
date                                                            
2015-01-28                                                      
2015-03-18                                                      
2015-04-29  Michael Dotsey, Craig S. Hakkio, Evan F. Koeni...   
2015-06-17                                                      
2015-07-29  Glenn Follette and Paul A. Smith, Assistant Di...   

           

## ordinal_encode

In [47]:
from sklearn.preprocessing import OrdinalEncoder

In [48]:
def ordinal_encode(df, column):
    
    ordinal_encoder = OrdinalEncoder()
    df[f'{column}_encoded'] = ordinal_encoder.fit_transform(df[[column]]).astype(int)
    
    return df

In [49]:
pairing_df = ordinal_encode (pairing_df, 'decision')
print(pairing_df['decision_encoded'].head())

date
2015-01-28    1
2015-03-18    1
2015-04-29    1
2015-06-17    1
2015-07-29    1
Name: decision_encoded, dtype: int64


## FinBERT_vectorization and finalize_df

In [53]:
from transformers import AutoTokenizer, AutoModel
import torch

In [50]:
def FinBERT_vectorizaion(text):
    
    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
    model = AutoModel.from_pretrained("ProsusAI/finbert")
    
    # Ensure text is not empty or NaN
    if pd.isna(text) or text.strip() == "":
        return np.zeros((768,),dtype=np.float32)  # Return a zero-vector if input is empty or NaN
    
    # Proceed with tokenization and embedding generation
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    
    return outputs.last_hidden_state[:, 0, :].squeeze().numpy().astype(np.float32)  

In [51]:
def finalize_df(df):
    
    # Vectorize statements and minutes
    df['statement_vectorized'] = df['grouped_statements'].apply(lambda x: np.array(FinBERT_vectorizaion(str(x)), dtype=np.float32))
    df['minutes_vectorized'] = df['grouped_minutes'].apply(lambda x: np.array(FinBERT_vectorizaion(str(x)), dtype=np.float32))

    # Combine the two vectors into one
    df['combined_vectorization'] = df.apply(
        lambda row: np.hstack((
            row['statement_vectorized'],  # Ensure this is a numpy array
            row['minutes_vectorized'] if isinstance(row['minutes_vectorized'], np.ndarray) else np.zeros(768, dtype=np.float32)
        )),
        axis=1
    )
    
    # Make sure the 'combined_vectorization' column is stored as a numpy array of dtype float32
    df['combined_vectorization'] = df['combined_vectorization'].apply(lambda x: np.array(x, dtype=np.float32))
    
    # Debugging: Check data type after processing
    print(df['combined_vectorization'].apply(type).head(3))  # Should show <class 'numpy.ndarray'>
    
    return df

In [54]:
final_df = finalize_df(pairing_df)

2025-02-10 13:19:31.944047: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-10 13:19:32.547710: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-10 13:19:32.607173: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-02-10 13:19:32.607191: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

date
2015-01-28    <class 'numpy.ndarray'>
2015-03-18    <class 'numpy.ndarray'>
2015-04-29    <class 'numpy.ndarray'>
Name: combined_vectorization, dtype: object


In [64]:
print(final_df['combined_vectorization'].head())

date
2015-01-28    [-0.24246012, 0.67152435, -0.20416914, -0.2890...
2015-03-18    [-0.07346612, 0.82941, -0.24438372, -0.5677416...
2015-04-29    [-0.092187196, -0.2836279, -0.6326303, -0.3542...
2015-06-17    [0.060003556, 0.84255415, -0.253191, -0.209660...
2015-07-29    [0.06376937, -0.35077733, -0.35831827, -0.2351...
Name: combined_vectorization, dtype: object


## train_test_split

In [65]:
from sklearn.model_selection import train_test_split

In [66]:
def custom_train_test_split(df, test_size=0.2, random_state=42):
    """ Splits the dataset into training and testing sets. """
   
    # Convert stringified arrays back to real NumPy arrays
    df['combined_vectorization'] = df['combined_vectorization'].apply(lambda x: np.array(ast.literal_eval(x), dtype=np.float32) if isinstance(x, str) else x)

    # Stack the arrays to create the feature matrix
    X = np.vstack(df['combined_vectorization'].values)
    y = df['decision_encoded'].values

    # Split the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    return X_train, X_test, y_train, y_test  # ✅ Returns only 4 values

In [67]:
X_train, X_test, y_train, y_test = custom_train_test_split(final_df, test_size=0.2, random_state=42)

In [77]:
print(X_train)

[[-0.4603853   0.1941262  -0.56941366 ... -0.11998674  1.0199955
   0.5946898 ]
 [ 0.30797532 -0.3074921  -0.43068966 ...  0.05326387  0.83187574
   0.34394997]
 [-0.22082192  0.5572233   0.25929168 ... -0.17048459  0.4922418
   0.46881706]
 ...
 [-0.9463986   0.479746   -0.1653099  ... -0.50802386 -0.38364428
   0.5584152 ]
 [-0.12968168 -0.0050654  -0.41801387 ... -0.47364542  0.03962266
   0.66211003]
 [-0.01522618  0.37421858 -0.64247054 ... -0.06257439  0.06237637
   0.34195045]]


In [79]:
print(y_train)

[1 1 1 1 1 2 1 1 1 2 2 1 0 1 1 1 2 2 2 1 1 2 1 2 0 1 1 1 0 2 1 1 2 2 0 1 1
 1 2 2 1 1 1 2 0 2 1 2 1 1 1 1 2 1 2 1 1]


## class_weighting

In [82]:
def class_weighting (y_train):

    # Convert y_train to a Pandas Series to use value_counts
    y_train = pd.Series(y_train)
    
    # Count occurrences in y_train
    class_counts = y_train.value_counts().sort_index().values  # Ensure ordering is correct

    # Compute class weights
    class_weights = torch.tensor([1 / count for count in class_counts], dtype=torch.float32)
    
    return class_weights


In [83]:
class_weights = class_weighting(y_train)

In [84]:
print(class_weights)

tensor([0.2000, 0.0294, 0.0556])


## tensor_conversion

In [90]:
def tensor_conversion(X_train, y_train):
    
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    # BiLSTM will need 3D tensors. our tensors only have 2Ds for X
    # conversely, only 1D tensor for y with integer data, and not float
    X_train_tensor = X_train_tensor.unsqueeze(1)
    
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    
    return X_train_tensor, y_train_tensor

In [91]:
X_train_tensor, y_train_tensor = tensor_conversion(X_train, y_train)

In [92]:
print(X_train_tensor.shape)
print(y_train_tensor.shape)

torch.Size([57, 1, 1536])
torch.Size([57])


In [93]:
X_test_tensor, y_test_tensor = tensor_conversion(X_test, y_test)
print(X_test_tensor.shape)
print(y_test_tensor.shape)

torch.Size([15, 1, 1536])
torch.Size([15])


## initialize_model

In [96]:
import numpy as np

from sklearn.model_selection import KFold

import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.metrics import accuracy_score

from params import HIDDEN_DIM, OUTPUT_DIM, K, LEARNING_RATE, RANDOM_STATE, EPOCHS

In [97]:
INPUT_DIM = X_train.shape[1]  # Same as embedding size
HIDDEN_DIM = 128
OUTPUT_DIM = 3  # For multiclass classification (up, down, no change)
K = 5 
LEARNING_RATE = 0.001
EPOCHS = 10
RANDOM_STATE = 42

In [98]:
class BiLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Multiply by 2 for bidirectional
        self.relu = nn.ReLU()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_out = lstm_out[:, -1, :]  # Take the last time step's output
        return self.fc(self.relu(last_out))


def initialize_model(X_train, hidden_dim = HIDDEN_DIM, output_dim = OUTPUT_DIM):
    
    input_dim = X_train.shape[1]
    
    model = BiLSTM(input_dim, hidden_dim, output_dim)
    
    return model

In [99]:
model = initialize_model(X_train, HIDDEN_DIM, OUTPUT_DIM)
print(model)

BiLSTM(
  (lstm): LSTM(1536, 128, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=3, bias=True)
  (relu): ReLU()
)


## compile_model

In [100]:
def compile_model(y_train, model, learning_rate=LEARNING_RATE):
    
    class_weights = class_weighting (y_train)
    
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    return criterion, optimizer

In [101]:
criterion, optimizer = compile_model(y_train, model, learning_rate=LEARNING_RATE)

In [102]:
print(criterion, optimizer)

CrossEntropyLoss() Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0
)


## evaluate_model and train_model

In [103]:
def evaluate_model(model, X_test_tensor, y_test_tensor):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient computation for efficiency
        outputs = model(X_test_tensor)  # Get the model's output

        logits = outputs  # These are the raw predictions (logits)

        # Get the predicted class (highest logit)
        predicted_labels = torch.argmax(logits, dim=1)
        
    # Compute accuracy by comparing predicted labels to true labels
    accuracy = accuracy_score(y_test_tensor.numpy(), predicted_labels.numpy())
    
    return accuracy

In [104]:
accuracy = evaluate_model(model, X_test_tensor, y_test_tensor)
print(accuracy)

0.8


In [107]:
def train_model(X_train_tensor, y_train_tensor, y_train, n_splits=K):
    
    kf = KFold(n_splits=K, shuffle=True, random_state=RANDOM_STATE)
    
    fold_accuracies =[]
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_tensor, y_train_tensor)):
        print(f"Training fold {fold + 1}/{K}...")

        # Split data into training and validation sets for this fold
        X_train_fold, X_val_fold = X_train_tensor[train_idx], X_train_tensor[val_idx]
        y_train_fold, y_val_fold = y_train_tensor[train_idx], y_train_tensor[val_idx]

        # Convert to PyTorch tensors
        X_train_fold_tensor = torch.tensor(X_train_fold, dtype=torch.float32)
        y_train_fold_tensor = torch.tensor(y_train_fold, dtype=torch.long)
        X_val_fold_tensor = torch.tensor(X_val_fold, dtype=torch.float32)
        y_val_fold_tensor = torch.tensor(y_val_fold, dtype=torch.long)

        # Initialize the model, criterion, and optimizer
        model = initialize_model(X_train, hidden_dim=128, output_dim=3)
        criterion, optimizer = compile_model(y_train, model, learning_rate = LEARNING_RATE)

        # Train the model on this fold
        model.train()  # Set model to training mode
        for epoch in range(EPOCHS):
            optimizer.zero_grad()
            outputs = model(X_train_fold_tensor)
            loss = criterion(outputs, y_train_fold_tensor)
            loss.backward()
            optimizer.step()
            
        # Evaluate the model on the validation set
        accuracy = evaluate_model(model, X_val_fold_tensor, y_val_fold_tensor)
        print(f"Fold {fold + 1} Accuracy: {accuracy:.4f}")

        # Save the accuracy for this fold
        fold_accuracies.append(accuracy)

        # Calculate average accuracy across all folds
        average_accuracy = np.mean(fold_accuracies)
        print(f"\nAverage Cross-Validation Accuracy: {average_accuracy:.4f}")

    return model, average_accuracy

In [108]:
model, average_accuracy = train_model(X_train_tensor, y_train_tensor, y_train, n_splits=K)
print(model, average_accuracy)

Training fold 1/5...


/tmp/ipykernel_48266/161865244.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_fold_tensor = torch.tensor(X_train_fold, dtype=torch.float32)
/tmp/ipykernel_48266/161865244.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_fold_tensor = torch.tensor(y_train_fold, dtype=torch.long)
/tmp/ipykernel_48266/161865244.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_fold_tensor = torch.tensor(X_val_fold, dtype=torch.float32)
/tmp/ipykernel_48266/161865244.py:18: UserWarning: To copy construct from a tensor, it is

Fold 1 Accuracy: 0.4167

Average Cross-Validation Accuracy: 0.4167
Training fold 2/5...
Fold 2 Accuracy: 0.4167

Average Cross-Validation Accuracy: 0.4167
Training fold 3/5...


/tmp/ipykernel_48266/161865244.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_fold_tensor = torch.tensor(X_train_fold, dtype=torch.float32)
/tmp/ipykernel_48266/161865244.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_fold_tensor = torch.tensor(y_train_fold, dtype=torch.long)
/tmp/ipykernel_48266/161865244.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_fold_tensor = torch.tensor(X_val_fold, dtype=torch.float32)
/tmp/ipykernel_48266/161865244.py:18: UserWarning: To copy construct from a tensor, it is

Fold 3 Accuracy: 0.3636

Average Cross-Validation Accuracy: 0.3990
Training fold 4/5...
Fold 4 Accuracy: 0.8182

Average Cross-Validation Accuracy: 0.5038
Training fold 5/5...


/tmp/ipykernel_48266/161865244.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_fold_tensor = torch.tensor(X_train_fold, dtype=torch.float32)
/tmp/ipykernel_48266/161865244.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_fold_tensor = torch.tensor(y_train_fold, dtype=torch.long)
/tmp/ipykernel_48266/161865244.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_fold_tensor = torch.tensor(X_val_fold, dtype=torch.float32)
/tmp/ipykernel_48266/161865244.py:18: UserWarning: To copy construct from a tensor, it is

Fold 5 Accuracy: 0.5455

Average Cross-Validation Accuracy: 0.5121
BiLSTM(
  (lstm): LSTM(1536, 128, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=256, out_features=3, bias=True)
  (relu): ReLU()
) 0.5121212121212121
